In [24]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [25]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [26]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [27]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,140.000000,140.000000
mean,0.179244,0.540043
std,0.150606,2.141017
min,0.000000,0.007332
25%,0.000000,0.096924
50%,0.181818,0.140086
75%,0.255682,0.277923
max,0.750000,18.006205


In [28]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [29]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1166
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.733
Time:                        23:37:31   Log-Likelihood:                -304.67
No. Observations:                 140   AIC:                             613.3
Df Residuals:                     138   BIC:                             619.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6141      0.283      2.172      0.032       0.055       1.173
X             -0.4131      1.210     -0.342      0.733      -2.805       1.979
==============================================================================
Omnibus:                      238.333   Durbin-Watson:                   0.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22075.281
Skew:                           7.701   Prob(JB):                         0.00
Kurtosis:                      62.557   Cond. No.                         6.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [31]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2424
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.623
Time:                        23:37:31   Log-Likelihood:                -294.18
No. Observations:                 140   AIC:                             592.4
Df Residuals:                     138   BIC:                             598.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7279      0.262      2.774      0.006       0.209       1.247
X              0.5526      1.122      0.492      0.623      -1.667       2.772
==============================================================================
Omnibus:                      244.266   Durbin-Watson:                   0.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24867.894
Skew:                           8.029   Prob(JB):                         0.00
Kurtosis:                      66.287   Cond. No.                         6.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [33]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [34]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [35]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [36]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [37]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [38]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [39]:
data_sample.describe()

,X,Y
count,122.000000,122.000000
mean,0.189160,0.157972
std,0.141897,0.100272
min,0.000000,0.007332
25%,0.090909,0.089434
50%,0.200000,0.129841
75%,0.272727,0.202422
max,0.545455,0.456443


In [40]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.789
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.184
Time:                        23:37:32   Log-Likelihood:                 108.88
No. Observations:                 122   AIC:                            -213.8
Df Residuals:                     120   BIC:                            -208.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1418      0.015      9.377      0.000       0.112       0.172
X              0.0856      0.064      1.337      0.184      -0.041       0.212
==============================================================================
Omnibus:                       23.216   Durbin-Watson:                   0.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.577
Skew:                           1.171   Prob(JB):                     3.78e-07
Kurtosis:                       3.581   Cond. No.                         7.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
bX = data_sample["X"] * result.params["X"]

In [42]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [43]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.522
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.220
Time:                        23:37:32   Log-Likelihood:                 163.05
No. Observations:                 122   AIC:                            -322.1
Df Residuals:                     120   BIC:                            -316.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0661      0.010      6.818      0.000       0.047       0.085
X              0.0507      0.041      1.234      0.220      -0.031       0.132
==============================================================================
Omnibus:                       30.729   Durbin-Watson:                   1.206
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.887
Skew:                           1.327   Prob(JB):                     2.95e-10
Kurtosis:                       4.262   Cond. No.                         7.33
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""